In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.regularizers import l1_l2
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


# Data preprocessing
def preprocess_data(df):
    try:
        # Process temporal features
        df['Seconds'] = df.index.map(pd.Timestamp.timestamp)
        
        day = 24 * 60 * 60
        year = (365.2425) * day
        
        # Add cyclical time features
        df['Day sin'] = np.sin(df['Seconds'] * (2 * np.pi / day))
        df['Day cos'] = np.cos(df['Seconds'] * (2 * np.pi / day))
        df['Year sin'] = np.sin(df['Seconds'] * (2 * np.pi / year))
        df['Year cos'] = np.cos(df['Seconds'] * (2 * np.pi / year))
        
        # # Add rolling statistics
        # df['rolling_mean_6h'] = df.iloc[:, 0].rolling(window=6).mean()
        # df['rolling_std_6h'] = df.iloc[:, 0].rolling(window=6).std()
        # df['rolling_max_6h'] = df.iloc[:, 0].rolling(window=6).max()
        
        # # Add lag features
        # for i in [1, 3, 6, 12]:
        #     df[f'lag_{i}'] = df.iloc[:, 0].shift(i)
        
        df = df.bfill()
        df = df.drop(['Seconds'], axis=1)
        
        return df
    except Exception as e:
        print(f"Error in preprocess_data: {e}")
        raise

def df_to_X_y(df, window_size=24):
    try:
        df_as_np = df.to_numpy()
        X = []
        y = []
        
        for i in range(len(df_as_np) - window_size):
            row = [r for r in df_as_np[i: i + window_size]]
            X.append(row)
            
            label = df_as_np[i + window_size][0]
            y.append(label)
        
        return np.array(X), np.array(y)
    except Exception as e:
        print(f"Error in df_to_X_y: {e}")
        raise

def split_time_series_data(X, y, train_ratio=0.8, val_ratio=0.1):
    try:
        n = len(X)
        train_end = int(n * train_ratio)
        val_end = train_end + int(n * val_ratio)

        X_train, y_train = X[:train_end], y[:train_end]
        X_val, y_val = X[train_end:val_end], y[train_end:val_end]
        X_test, y_test = X[val_end:], y[val_end:]

        return X_train, X_val, X_test, y_train, y_val, y_test
    except Exception as e:
        print(f"Error in split_time_series_data: {e}")
        raise

def standardize_features(X_train, X_val, X_test):
    """
    Standardize all features in the dataset while maintaining the 3D structure
    (samples, timesteps, features).
    """
    num_samples, num_timesteps, num_features = X_train.shape
    
    # Reshape to 2D for StandardScaler (combine samples & timesteps)
    X_train_reshaped = X_train.reshape(-1, num_features)
    
    # Fit the scaler on training data only
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_reshaped)

    # Transform validation and test sets
    X_val_scaled = scaler.transform(X_val.reshape(-1, num_features))
    X_test_scaled = scaler.transform(X_test.reshape(-1, num_features))

    # Reshape back to original 3D shape
    X_train = X_train_scaled.reshape(num_samples, num_timesteps, num_features)
    X_val = X_val_scaled.reshape(X_val.shape[0], X_val.shape[1], num_features)
    X_test = X_test_scaled.reshape(X_test.shape[0], X_test.shape[1], num_features)

    return X_train, X_val, X_test, scaler

# Model architectures with regularization and dropout
def create_lstm_model(input_shape, dropout_rate=0.3):
    try:
        model = Sequential([
            InputLayer(shape=input_shape),
            
            LSTM(
                128, 
                return_sequences=True, 
                kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4),
                recurrent_regularizer=l1_l2(l1=1e-5, l2=1e-4)
            ),
            BatchNormalization(),
            Dropout(dropout_rate),
            
            LSTM(
                64, 
                return_sequences=False,
                kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)
            ),
            BatchNormalization(),
            Dropout(dropout_rate),
            
            Dense(
                32, 
                activation='relu',
                kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)
            ),
            Dropout(dropout_rate),
            
            Dense(1, activation='linear')
        ])
        return model
    except Exception as e:
        print(f"Error in create_lstm_model: {e}")
        raise

def create_gru_model(input_shape, dropout_rate=0.3):
    try:
        model = Sequential([
            InputLayer(shape=input_shape),
            
            GRU(
                128, 
                return_sequences=True,
                kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)
            ),
            BatchNormalization(),
            Dropout(dropout_rate),
            
            GRU(
                64, 
                return_sequences=False,
                kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)
            ),
            BatchNormalization(),
            Dropout(dropout_rate),
            
            Dense(
                32, 
                activation='relu',
                kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)
            ),
            Dropout(dropout_rate),
            
            Dense(1, activation='linear')
        ])
        return model
    except Exception as e:
        print(f"Error in create_gru_model: {e}")
        raise

def create_cnn_model(input_shape, dropout_rate=0.3):
    try:
        model = Sequential([
            InputLayer(shape=input_shape),
            
            Conv1D(
                filters=128, 
                kernel_size=3, 
                padding='same', 
                activation='relu',
                kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)
            ),
            BatchNormalization(),
            MaxPooling1D(pool_size=2),
            Dropout(dropout_rate),
            
            Conv1D(
                filters=64, 
                kernel_size=3, 
                padding='same', 
                activation='relu',
                kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)
            ),
            BatchNormalization(),
            MaxPooling1D(pool_size=2),
            Dropout(dropout_rate),
            Flatten(),
            
            Dense(
                32, 
                activation='relu',
                kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)
            ),
            Dropout(dropout_rate),
            
            Dense(1, activation='linear')
        ])
        return model
    except Exception as e:
        print(f"Error in create_cnn_model: {e}")
        raise

# Weighted mean squared error
def weighted_mse(y_true, y_pred):
    try:
        weights = tf.where(y_pred < y_true, 2.0, 1.0) # Penalize underestimation more heavily (false negatives)
        squared_difference = tf.square(y_true - y_pred)
        return tf.reduce_mean(weights * squared_difference)
    except Exception as e:
        print(f"Error in weighted_mse: {e}")
        raise

# Custom loss function to penalize false negatives more heavily
def custom_loss(y_true, y_pred):
    try:
        squared_difference = tf.square(y_true - y_pred)
        mse = tf.reduce_mean(squared_difference, axis=-1)
        
        # Penalize underestimation more heavily (false negatives)
        penalty = tf.where(y_pred < y_true, 2.0, 1.0)
        return mse * penalty
    except Exception as e:
        print(f"Error in custom_loss: {e}")
        raise


# Training function with callbacks
def train_model(model, X_train, y_train, X_val, y_val, model_path, 
                batch_size=32, epochs=5, patience=3, loss_function='mse', optimizer_function='adam', learning_rate=1e-3):
    try:
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True, start_from_epoch=3),
            ModelCheckpoint(model_path, save_best_only=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=patience, min_lr=1e-6)
        ]
        
        # Select the optimizer based on the input parameter
        if optimizer_function == 'adamw':
            optimizer = AdamW(learning_rate=learning_rate, weight_decay=1e-4)
        elif optimizer_function == 'adam':
            optimizer = Adam(learning_rate=learning_rate)
        
        # Select the loss function based on the input parameter
        if loss_function == 'weighted_mse':
            loss = weighted_mse
        elif loss_function == 'custom':
            loss = custom_loss
        
        model.compile(
            optimizer=optimizer,
            loss=loss,
            metrics=[
                'mse', 
                'mae',
                'root_mean_squared_error',
            ]
        )
        
        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            batch_size=batch_size,
            epochs=epochs,
            callbacks=callbacks,
            verbose=1
        )
        return history
    except Exception as e:
        print(f"Error in train_model: {e}")
        raise


# Evaluation function with focus on false negatives
# def evaluate_model(model, X_test, y_test, threshold=0.5):
#     try:
#         predictions = model.predict(X_test)
        
#         # Compute MSE and MAE
#         mse = np.mean((y_test - predictions.flatten())**2)
#         mae = np.mean(np.abs(y_test - predictions.flatten()))
        
#         # Convert to binary labels
#         binary_actual = y_test > threshold
#         binary_pred = predictions.flatten() > threshold
        
#         # Compute false negatives and FNR
#         false_negatives = np.sum((binary_actual == True) & (binary_pred == False))
#         total_positives = np.sum(binary_actual)
#         false_negative_rate = false_negatives / total_positives if total_positives > 0 else 0.0
        
#         return {
#             'mse': mse,
#             'mae': mae,
#             'false_negative_rate': false_negative_rate
#         }
#     except Exception as e:
#         print(f"Error in evaluate_model: {e}")
#         raise

def evaluate_model(model, X_test, y_test, threshold=0.5):
    """
    Evaluates a time series forecasting model with focus on false negatives
    and additional metrics (CSI, POD, FAR, Precision, Recall, F1-score)
    using TensorFlow.

    Args:
        model (tf.keras.Model): Trained TensorFlow Keras model.
        X_test (np.ndarray or tf.Tensor): Test data features.  (Assumed already a tensor or convertible)
        y_test (np.ndarray or tf.Tensor): Test data labels (actual values). (Assumed already a tensor or convertible)
        threshold (float): Threshold to define a cloudburst event.

    Returns:
        dict: A dictionary of evaluation metrics (all TensorFlow tensors).
    """
    try:
        # Ensure X_test and y_test are TensorFlow tensors
        X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
        y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

        predictions = model.predict(X_test)  # model.predict returns numpy array

        # Convert predictions to a TensorFlow tensor
        predictions = tf.convert_to_tensor(predictions.flatten(), dtype=tf.float32)

        # Compute MSE and MAE (using TensorFlow)
        mse = tf.reduce_mean((y_test - predictions)**2)
        mae = tf.reduce_mean(tf.abs(y_test - predictions))

        # Compute RMSE (using TensorFlow)
        rmse = tf.sqrt(mse)  # RMSE is the square root of MSE

        # Convert to binary labels (using TensorFlow)
        binary_actual = tf.cast(y_test > threshold, tf.float32)
        binary_pred = tf.cast(predictions > threshold, tf.float32)

        # Calculate True Positives, False Positives, False Negatives, True Negatives
        tp = tf.reduce_sum(binary_actual * binary_pred)
        fp = tf.reduce_sum((1 - binary_actual) * binary_pred)
        fn = tf.reduce_sum(binary_actual * (1 - binary_pred))
        tn = tf.reduce_sum((1 - binary_actual) * (1 - binary_pred))

        # Handle division by zero
        epsilon = 1e-7  # Small constant to prevent division by zero

        # Compute additional metrics (using TensorFlow)
        csi = tp / (tp + fn + fp + epsilon)
        pod = tp / (tp + fn + epsilon)
        far = fp / (tp + fp + epsilon)
        precision = tp / (tp + fp + epsilon)
        recall = tp / (tp + fn + epsilon)  # This is the same as POD
        f1 = 2 * (precision * recall) / (precision + recall + epsilon)

        # Compute false negative rate (using TensorFlow)
        total_positives = tf.reduce_sum(tf.cast(y_test > threshold, tf.float32))
        false_negatives = tf.cast(fn, tf.float32) #Cast to float32 for division
        false_negative_rate = false_negatives / (total_positives + epsilon)

        return {
            'mse': mse,
            'mae': mae,
            'rmse': rmse,
            'false_negative_rate': false_negative_rate,
            'CSI': csi,
            'POD': pod,
            'FAR': far,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1
        }
    except Exception as e:
        print(f"Error in evaluate_model: {e}")
        raise #Re-raise the exception

def plot_predictions(model, X_data, y_data, label, start=50, end=500, ylabel='Rainfall (mm)', title_suffix=''):
    # Make predictions
    predictions = model.predict(X_data).flatten()

    # Create a DataFrame to store results
    results_df = pd.DataFrame(data={f'{label} Predictions': predictions, 'Actual Values': y_data})
    print(results_df)

    # Plot the predictions and actual values
    plt.figure(figsize=(10, 6))
    plt.plot(results_df[f'{label} Predictions'][start:end], label=f'{label} Predictions', color='blue', linestyle='-')
    plt.plot(results_df['Actual Values'][start:end], label='Actual Values', color='orange', linestyle='--')

    # Add labels and title
    plt.xlabel('Time Stamps', fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.title(f'{label} Predictions vs Actual Values {title_suffix}', fontsize=14)
    plt.legend(loc='upper right')
    plt.grid(True)
    # plt.show()

    return results_df

In [2]:
# Main execution
def main():
    try:
        # Load and preprocess data
        df = pd.read_csv('../artifacts/dataset/01-hourly_historical_analyzed_data.csv')
        df = df.drop(columns=['hour', 'day', 'month', 'year'])
        
        # # testing 0.1
        # print(f"df Dataframe: {df.head()}")
        
        # Convert to datetime index
        df1 = df.copy()
        df1.index = pd.to_datetime(df1['time'], format='%Y-%m-%d %H:%M:%S')
        
        # # testing 0.2
        # print(f"df1 Dataframe: {df1.head()}")
        
        print("********DATA INGESTION COMPLETE********")
        
        # Extract rain data
        rain = df1['rain']
        rain_df = pd.DataFrame(rain)
        
        # Preprocess data with enhanced features
        rain_df = preprocess_data(rain_df)
        rain_df = rain_df.drop(['rain'], axis=1)
        
        # # testing 0.3
        # print(f"rain_df Dataframe: {rain_df.head()}")
        
        processed_df = pd.concat([df1, rain_df], axis=1)
        processed_df = processed_df.drop(['time'], axis=1)
        
        # # testing 0.4
        # print(f"processed_df Dataframe: {processed_df.head()}")
        
        # Create sequences
        X, y = df_to_X_y(processed_df, window_size=24)
        
        # # testing 0.5
        # print(f"X shape: {X.shape}, y shape: {y.shape}")
        
        # Split data
        X_train, X_val, X_test, y_train, y_val, y_test = split_time_series_data(X, y)
        
        # # testing 0.6
        # print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
        # print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
        # print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
        
        # Scale the data
        X_train, X_val, X_test, X_scaler = standardize_features(X_train, X_val, X_test)
        
        # # testing 0.7
        # print("Standardization complete")
        
        print("********DATA PREPROCESSING COMPLETE********")
        
        # Define optimizers and loss functions to iterate over
        optimizers = ['adam', 'adamw']
        loss_functions = ['weighted_mse', 'custom']
        
        # for loss_function in loss_functions:
        
        #     # Create a directory for the current loss function
        #     results_dir = f'../artifacts/results/cycle_2/test_3/{loss_function}'
        #     os.makedirs(results_dir, exist_ok=True)
        
        #     # Train models
        #     models = {
        #         'lstm': create_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2])),
        #         'gru': create_gru_model(input_shape=(X_train.shape[1], X_train.shape[2])),
        #         'cnn': create_cnn_model(input_shape=(X_train.shape[1], X_train.shape[2])),
        #     }
            
        #     results = {}
        #     for name, model in models.items():
        #         print(f"\nTraining {name.upper()} model with {loss_function.upper()} loss...")
                
        #         try:
        #             history = train_model(
        #                 model, X_train, y_train, X_val, y_val,
        #                 f'../artifacts/models/cycle_2/test_3/model_{name}_{loss_function}.keras', 
        #                 epochs=5, loss_function=loss_function, learning_rate=1e-3
        #             )
                    
        #             print("********MODEL TRAINING COMPLETE********")
        #         except Exception as e:
        #             print(f"Error training {name.upper()} model: {e}")
        #             continue
        
        for optimizer_function in optimizers:
            for loss_function in loss_functions:
                
                # Create a directory for the current optimizer-loss function combination
                results_dir = f'../artifacts/results/cycle_2/test_3/{optimizer_function}_{loss_function}'
                os.makedirs(results_dir, exist_ok=True)

                # Train models
                models = {
                    'lstm': create_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2])),
                    'gru': create_gru_model(input_shape=(X_train.shape[1], X_train.shape[2])),
                    'cnn': create_cnn_model(input_shape=(X_train.shape[1], X_train.shape[2])),
                }

                results = {}
                for name, model in models.items():
                    print(f"\nTraining {name.upper()} model with {optimizer_function.upper()} optimizer and {loss_function.upper()} loss...")

                    try:
                        history = train_model(
                            model, X_train, y_train, X_val, y_val,
                            f'../artifacts/models/cycle_2/test_3/model_{name}_{optimizer_function}_{loss_function}.keras', 
                            epochs=5, loss_function=loss_function, optimizer_function=optimizer_function, learning_rate=1e-3
                        )

                        print("********MODEL TRAINING COMPLETE********")
                    except Exception as e:
                        print(f"Error training {name.upper()} model: {e}")
                        continue
                
                    try:
                        results[name] = evaluate_model(model, X_test, y_test)
                    except Exception as e:
                        print(f"Error evaluating {name.upper()} model: {e}")
                        continue
                    
                    # Save training history to CSV
                    history_df = pd.DataFrame(history.history)
                    history_df.to_csv(os.path.join(results_dir, f'{name}_history.csv'), index=False)
                    
                    # Plot training history
                    plt.figure(figsize=(10, 6))
                    plt.plot(history.history['loss'], label='Training Loss')
                    plt.plot(history.history['val_loss'], label='Validation Loss')
                    plt.title(f'{name.upper()} Model Training History')
                    plt.xlabel('Epoch')
                    plt.ylabel('Loss')
                    plt.legend()
                    plt.savefig(os.path.join(results_dir, f'{name}_training_history.png'))
                    # plt.show()
                    plt.close()  # Close the plot to free memory
                    
                    # Save evaluation results to a text file
                    with open(os.path.join(results_dir, f'{name}_evaluation.txt'), 'w') as f:
                        for metric_name, value in results[name].items():
                            f.write(f"{metric_name}: {value:.4f}\n")
                    
                    # Plot predictions for Train, Val, and Test datasets and save the plots
                    for dataset, data, true_values in zip(['Train', 'Val', 'Test'], 
                                                        [X_train, X_val, X_test], 
                                                        [y_train, y_val, y_test]):
                        plot_predictions(
                            model=model, 
                            X_data=data, 
                            y_data=true_values, 
                            label=name + ' ' + dataset, 
                            start=100, 
                            end=500
                        )
                        plt.savefig(os.path.join(results_dir, f'{name}_{dataset.lower()}_predictions.png'))
                        # plt.show()
                        plt.close()  # Close the plot to free memory
                        
                    print("********MODEL EVALUATION COMPLETE********")
            
            print(f"Results for loss function '{loss_function}' saved in '{results_dir}'.")
            
        # Return results
        return results
    except Exception as e:
        print(f"Error in main: {e}")
        raise

if __name__ == "__main__":
    try:
        results = main()
        print("\nModel Evaluation Results:")
        for model_name, metrics in results.items():
            print(f"\n{model_name.upper()}:")
            for metric_name, value in metrics.items():
                print(f"{metric_name}: {value:.4f}")
    except Exception as e:
        print(f"Unhandled error in execution: {e}")

********DATA INGESTION COMPLETE********
********DATA PREPROCESSING COMPLETE********

Training LSTM model with ADAM optimizer and WEIGHTED_MSE loss...
Epoch 1/5
5480/5480 ━━━━━━━━━━━━━━━━━━━━ 179s 32ms/step - loss: 72.0343 - mae: 4.8271 - mse: 43.0378 - root_mean_squared_error: 6.2710 - val_loss: 2.0916 - val_mae: 0.9608 - val_mse: 1.5836 - val_root_mean_squared_error: 1.2584 - learning_rate: 0.0010
Epoch 2/5
5480/5480 ━━━━━━━━━━━━━━━━━━━━ 178s 33ms/step - loss: 14.8470 - mae: 2.5191 - mse: 10.5864 - root_mean_squared_error: 3.2520 - val_loss: 1.5929 - val_mae: 0.8436 - val_mse: 1.1963 - val_root_mean_squared_error: 1.0937 - learning_rate: 0.0010
Epoch 3/5
5480/5480 ━━━━━━━━━━━━━━━━━━━━ 177s 32ms/step - loss: 10.0476 - mae: 2.0553 - mse: 7.1633 - root_mean_squared_error: 2.6758 - val_loss: 1.5224 - val_mae: 0.7976 - val_mse: 1.1005 - val_root_mean_squared_error: 1.0490 - learning_rate: 0.0010
Epoch 4/5
5480/5480 ━━━━━━━━━━━━━━━━━━━━ 178s 32ms/step - loss: 7.9118 - mae: 1.8184 - mse: 5.6

## best performance now
1. GRU with *weighted loss* **(cycle_2/test_1)**
   * mse: 0.8157
   * mae: 0.6507
   * false_negative_rate: 0.0000
2. LSTM with *weighted loss* with *AdamW optimizer* **(cycle_2/test_3)**
   * mse: 0.7435
   * mae: 0.6114
   * rmse: 0.8623
   * false_negative_rate: 0.0000

## overall best performance till now
1. LSTM with *weighted loss* **(cycle_1/test_14)**
   * mse: 0.7585
   * mae: 0.6146
   * false_negative_rate: 0.0000

## All models have zero false negative rate: 
This means none of the models failed to predict rainfall events when they actually occurred, which is particularly important for weather forecasting.

## Low values: 
All the error metrics are relatively low, suggesting that all three models performed reasonably well.

## Training curves: 
All models show good convergence in their training histories, with both training and validation loss decreasing and stabilizing, indicating proper training without significant overfitting.

## Visual patterns: 
From the graphs, all models seem to follow the general pattern of the actual rainfall values, but they struggle with extreme values (particularly low rainfall values where the actual data shows near-zero measurements).